In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
tf.test.is_built_with_cuda()
print(tf.version.VERSION)

2.8.0


In [4]:
!unzip '/content/drive/MyDrive/dsdayolofiles.zip' -d '/content'

Archive:  /content/drive/MyDrive/dsdayolofiles.zip
 extracting: /content/config/__init__.py  
  inflating: /content/config/__pycache__/__init__.cpython-39.pyc  
  inflating: /content/config/__pycache__/yolov4_config.cpython-39.pyc  
  inflating: /content/config/yolov4_config.py  
  inflating: /content/data/source/source/aachen_000000_000019_leftImg8bit.png  
  inflating: /content/data/source/source/aachen_000001_000019_leftImg8bit.png  
  inflating: /content/data/source/source/aachen_000002_000019_leftImg8bit.png  
  inflating: /content/data/source/source/aachen_000003_000019_leftImg8bit.png  
  inflating: /content/data/source/source/aachen_000004_000019_leftImg8bit.png  
  inflating: /content/data/source/source/aachen_000005_000019_leftImg8bit.png  
  inflating: /content/data/source/source/aachen_000006_000019_leftImg8bit.png  
  inflating: /content/data/source/source/aachen_000007_000019_leftImg8bit.png  
  inflating: /content/data/source/source/aachen_000008_000019_leftImg8bit.png  

In [5]:
pip install mmcv

     |████████████████████████████████| 530 kB 24.7 MB/s 
     |████████████████████████████████| 190 kB 56.5 MB/s 
  Created wheel for mmcv: filename=mmcv-1.5.0-py2.py3-none-any.whl size=807180 sha256=18534538e0a2a7370fc688f44ec220f3611f707934825a19d85ad72aa70e7cc3
  Stored in directory: /root/.cache/pip/wheels/09/b4/88/0fee9e050a776e49b7290c2853e8ec002eeb55d7f4a40cf78f
Successfully built mmcv


In [6]:
from model import build_model
import eval_voc

In [7]:
import os
path='/content'
os.chdir(path)

In [8]:
# import required libraries
import io
import os

from collections import defaultdict
import tensorflow as tf
import numpy as np
from keras import backend as K
# from keras.layers import Conv2D, Input, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D, MaxPool2D
# from keras.layers.merge import add, concatenate
# from keras.models import Model
# from keras.layers import Activation


In [9]:
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from IPython.display import display, Image
from IPython.display import display
from PIL import Image



In [10]:
import torch
from torch.autograd import Function

# Autograd Function objects are what record operation history on tensors,
# and define formulas for the forward and backprop.

class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        # Store context for backprop
        ctx.alpha = alpha
        
        # Forward pass is a no-op
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        # Backward pass is just to -alpha the gradient
        output = grad_output.neg() * ctx.alpha

        # Must return same number as inputs to forward()
        return output, None

In [14]:
import torch.nn as nn
from model.build_model import Build_Model
from eval_voc import Evaluation
image_size=608
class DAN(nn.Module):
    def __init__(self):
        super().__init__()
        print('DAN class',35*'*')
        self.model = Build_Model(weight_path='weight/yolov4.weights')
        self.class_classifier = Evaluation(gpu_id=0, weight_path='weight/yolov4.weights', visiual=None, eval=False, mode='det')
        self.domain_classifier1 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, stride=1, padding=1),
            nn.Flatten(),
            nn.Linear(80*80, 100), nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 2),
            nn.LogSoftmax(dim=1)
        )
        self.domain_classifier2 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
            nn.Conv2d(256, 1, kernel_size=1, stride=1, padding=1),
            nn.Flatten(),
            nn.Linear(42*42, 100), nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 2),
            nn.LogSoftmax(dim=1)
        )
        self.domain_classifier3 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
            nn.Conv2d(512, 1, kernel_size=1, stride=1, padding=1),
            nn.Flatten(),
            nn.Linear(23*23, 100), nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 2),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x, imgpath, grl_lambda=0.1):
        print('forward function',35*'*')
        classp, atten, backbone = self.model(x)
        imgp0=imgpath[0]
        imgp1=imgpath[1]
        class_pred_a, box_pred_a, scores_a = self.class_classifier.detection(p=imgp0)
        class_pred_b, box_pred_b, scores_b = self.class_classifier.detection(p=imgp1)
        class_pred=[]
        class_pred.append(class_pred_a)
        class_pred.append(class_pred_b)
        box_pred=[]
        box_pred.append(box_pred_a)
        box_pred.append(box_pred_b)
        print('a',class_pred_a)
        print('size',len(class_pred_a))
        print('a',box_pred_a)
        print('size',len(box_pred_a))
        print('a', scores_a)
        for s in scores_a:
          if s>0.5:
            print(s)
        features1 = backbone[0]

        features2 = backbone[1]

        features3 = backbone[2]

        reverse_features1 = GradientReversalFn.apply(features1, grl_lambda)
        reverse_features2 = GradientReversalFn.apply(features2, grl_lambda)
        reverse_features3 = GradientReversalFn.apply(features3, grl_lambda)
        domain_pred1 = self.domain_classifier1(reverse_features1)
        domain_pred2 = self.domain_classifier2(reverse_features2)
        domain_pred3 = self.domain_classifier3(reverse_features3)
        return domain_pred1, domain_pred2, domain_pred3, class_pred

In [12]:
import torchvision.transforms as tvtf
import torchvision as tv
import torch
from torchvision import datasets
image_size = 608
batch_size = 2 #??

transform= tvtf.Compose([
    tvtf.Resize([image_size,image_size]),
    tvtf.ToTensor()
])

from torchvision.datasets import ImageFolder

class ImageFolderWithPaths(datasets.ImageFolder):

    def __getitem__(self, index):
  
        img, label = super(ImageFolderWithPaths, self).__getitem__(index)
        
        path = self.imgs[index][0]
        
        return (img, label ,path)

dataset_source = ImageFolderWithPaths('data/source/', transform=transform)
data_loader_source = torch.utils.data.DataLoader(dataset=dataset_source, batch_size=batch_size, shuffle=True, num_workers=2)
dataset_target = ImageFolderWithPaths('data/target/', transform=transform)
data_loader_target = torch.utils.data.DataLoader(dataset=dataset_target, batch_size=batch_size, shuffle=True, num_workers=2)
#The picture in cat folder corresponds to label 0 and dog corresponds to 1
print(dataset_source.class_to_idx)
print(dataset_target.class_to_idx)

#Paths of all pictures and corresponding labels
print(dataset_source.imgs)
print(dataset_target.imgs)

# ds_source = tv.datasets.ImageFolder('data/', transform=tf_source)
# dl_source = torch.utils.data.DataLoader(ds_source, batch_size)

# ds_target = tv.datasets.ImageFolder('data/', transform=tf_target)
# dl_target = torch.utils.data.DataLoader(ds_target, batch_size)

{'source': 0}
{'target': 0}
[('data/source/source/aachen_000000_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000001_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000002_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000003_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000004_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000005_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000006_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000007_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000008_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000009_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000010_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000011_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000012_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000013_000019_leftImg8bit.png', 0), ('data/source/source/aachen_000014_000019_leftImg8bit.png', 0), ('data/sour

In [8]:
# from torchvision.datasets import ImageFolder


# dataset = ImageFolder('data/')

# #The picture in cat folder corresponds to label 0 and dog corresponds to 1
# print(dataset.class_to_idx)

# #Paths of all pictures and corresponding labels
# print(dataset.imgs)
# dataset[0][0].size()
# print(data_loader)
# for i in range(10):
#         batch_x, batch_y = next(iter(data_loader))
#         print(np.shape(batch_x), batch_y)

In [9]:
# import matplotlib.pyplot as plt

# to_img = tvtf.ToPILImage()
# #0.2 and 0.4 are approximations of the standard deviation and mean
# a=to_img(dataset[0][0])
# plt.imshow(a)
# plt.axis('off')
# plt.show()

In [15]:
model = DAN()

x0_s, y0_s, p0_s = next(iter(data_loader_source))
x0_t, y0_t, p0_t = next(iter(data_loader_target))

print('source domain: ', x0_s.shape, y0_s.shape)
print('target domain: ', x0_t.shape, y0_t.shape)
print(p0_s[0])
model(x0_s, p0_s)
model(x0_t, p0_t)

DAN class ***********************************
load darknet weights :  weight/yolov4.weights
loading weight BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
loading weight Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
loading weight BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
loading weight Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
loading weight BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
loading weight Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
loading weight BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
loading weight Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
loading weight BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
loading weight Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
loading weight Ba

(tensor([[-1.0926, -0.4085],
         [-0.3192, -1.2973]], grad_fn=<LogSoftmaxBackward0>),
 tensor([[-0.3948, -1.1203],
         [-0.5274, -0.8919]], grad_fn=<LogSoftmaxBackward0>),
 tensor([[-0.8413, -0.5642],
         [-0.6307, -0.7598]], grad_fn=<LogSoftmaxBackward0>),
 [array([ 0,  0,  0, ..., 18, 18, 18], dtype=int32),
  array([ 0,  0,  0, ..., 18, 18, 18], dtype=int32)])

In [ ]:
import torch.optim as optim

lr = 1e-3   #???
n_epochs = 1

# Setup optimizer as usual
model = DAN()
optimizer = optim.Adam(model.parameters(), lr)

# Two losses functions this time
loss_fn_class = torch.nn.BCELoss()
loss_fn_domain = torch.nn.NLLLoss()

In [ ]:
batch_size = 2
dl_source = torch.utils.data.DataLoader(dataset_source, batch_size)
dl_target = torch.utils.data.DataLoader(dataset_target, batch_size)

# We'll train the same number of batches from both datasets
max_batches = min(len(dl_source), len(dl_target))

In [ ]:
for epoch_idx in range(n_epochs):
    print(f'Epoch {epoch_idx+1:04d} / {n_epochs:04d}', end='\n=================\n')
    dl_source_iter = iter(dl_source)
    dl_target_iter = iter(dl_target)

    for batch_idx in range(max_batches):
        optimizer.zero_grad()
        # Training progress and GRL lambda
        grl_lambda = 0.1

        # Train on source domain
        X_s, y_s = next(dl_source_iter)
        print('y_s is',y_s)
        y_s_domain = torch.ones(batch_size, dtype=torch.long) # generate source domain labels

        domain_pred1, domain_pred2, domain_pred3, class_pred = model(X_s, grl_lambda)
        loss_s_label = loss_fn_class(class_pred, y_s)
        loss_s_domain1 = loss_fn_domain(domain_pred1, y_s_domain)
        loss_s_domain2 = loss_fn_domain(domain_pred2, y_s_domain)
        loss_s_domain3 = loss_fn_domain(domain_pred3, y_s_domain)

        # Train on target domain
        X_t, _ = next(dl_target_iter) # ignore target domain class labels!
        y_t_domain = torch.zeros(batch_size, dtype=torch.long) # generate target domain labels

        domain_pred1, domain_pred2, domain_pred3, _= model(X_t, grl_lambda)
        loss_t_domain1 = loss_fn_domain(domain_pred1, y_t_domain)
        loss_t_domain2 = loss_fn_domain(domain_pred2, y_t_domain)
        loss_t_domain3 = loss_fn_domain(domain_pred3, y_t_domain)
        
        loss = loss_t_domain1 + loss_t_domain2 + loss_t_domain3 + loss_s_domain1 + loss_s_domain2 + loss_s_domain3 + loss_s_label
        loss.backward()
        optimizer.step()
        
        print(f'[{batch_idx+1}/{max_batches}] '
              f'class_loss: {loss_s_label.item():.4f} ' f's_domain_loss: {loss_s_domain1.item():.4f} '
              f't_domain_loss: {loss_t_domain1.item():.4f} ' f'grl_lambda: {grl_lambda:.3f} '
             )

Epoch 0001 / 0001
y_s is tensor([0, 0])
forward function ***********************************


KeyboardInterrupt: ignored